Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 1.53 ms, sys: 3.69 ms, total: 5.22 ms
Wall time: 13.1 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.80/10469/1 Dashboard: http://10.20.0.80:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 342 ms, sys: 12.2 ms, total: 354 ms
Wall time: 2.45 s


[1.3072801251855735,
 1.2944663431266008,
 1.5617791891076198,
 1.2708872321520417,
 1.4313412816326387,
 1.6471140733781016,
 1.0854043398425326,
 1.3052436281424817,
 1.9978333031307411,
 1.6627027039099067,
 1.4028230298713895,
 1.9602860167461453,
 1.9206294258140546,
 1.5888733937920971,
 1.1550125719161461,
 1.2197474845413099,
 1.4182700278475573,
 1.4390643253997117,
 1.907722781435762,
 1.3437244489600548,
 1.110413589433128,
 1.4919615791021175,
 1.1817673644824398,
 1.742284290491289,
 1.798963620399472,
 1.872271733129278,
 1.8538716418571328,
 1.2342735083583052,
 1.9237713084362427,
 1.2288747208421387,
 1.7059938611571568,
 1.2939923324971891,
 1.0682932890889631,
 1.7382816836740298,
 1.4199662701485334,
 1.9012824481901949,
 1.058263393289876,
 1.3665328690149403,
 1.282102613628148,
 1.8739673199974352,
 1.9971039879276518,
 1.7824172214578233,
 1.0825047240800834,
 1.0456739611065167,
 1.301328194695067,
 1.1661982261092283,
 1.1185131463019455,
 1.1688871095501898,


2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.3045808339293121,
 1.4754202192059873,
 1.8945328910441026,
 1.2533712411680895,
 1.7072957763169467,
 1.906580251415566,
 1.3096175654632032,
 1.9373529574345254,
 1.7460101908680703,
 1.2073700528721312,
 1.1792404835722141,
 1.9376614795712326,
 1.7170156218969503,
 1.6759261463496322,
 1.6037259803463426,
 1.1624101026290132,
 1.0492229479025275,
 1.7773851391756632,
 1.3518947456175503,
 1.9360394479015905,
 1.6610767816202827,
 1.5122149513607424,
 1.0018086466256344,
 1.423295999808293,
 1.5435217992025347,
 1.9627702931391513,
 1.1186688715259026,
 1.3530264673595802,
 1.9883549535114857,
 1.214741085266705,
 1.8662731536071044,
 1.7583732672786598,
 1.7513397121627459,
 1.782300745391396,
 1.066643226999858,
 1.2656719618380814,
 1.4919979142083122,
 1.2910575553440307,
 1.0412836884428194,
 1.0515389499871182,
 1.126489629960465,
 1.3299588831773368,
 1.014589540659987,
 1.2266978779203215,
 1.1330892621599962,
 1.4315359575631197,
 1.8329143385261446,
 1.638373173193851,


3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 278 ms, sys: 9.88 ms, total: 288 ms
Wall time: 2.2 s
